In [1]:
from amplpy import AMPL, modules


In [2]:
from scipy.optimize import linprog
import numpy as np
import pandas as pd
import plotly.express as px
import datetime


import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import timedelta
import datetime
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Battery specs :



- SOC: state of charge in [0,1]
- NEC: nominal energy capacity (Wh)
- EC: energy capacity (Wh)




# Schedule optimization functions :

In [55]:
def optimize_day_by_day(df) :
    """
    Returns a copy of the initial price data frame, with three additional columns :
    - schedule: Change in EC during the hour
    - capacity: Capacity at the end of the hour
    - SOC: SOC at the end of the hour 

    The schedule is optimized **for each day of the dataframe, considered separately.** 
    """
    NEC = 100000

    modules.load() # load all AMPL modules
    ampl = AMPL() # instantiate AMPL object
    ampl.read("ampl.mod")
    ampl.get_parameter("NEC").set_values([NEC])
    
    
    n_hours = len(df)
    if  n_hours % 24 != 0 :
        raise Exception("The dataframe should contain only full days (24 hours)")
    
    schedule = np.zeros(n_hours)
    
    ## first day starts with empty battery :
    initial_capacity = 0
    
    ## optimization done for each day :
    for day in range(0,n_hours//24) :

        
        price = df.price_euros_wh.iloc[day*24:(day+1)*24].to_numpy()
        ampl.get_parameter("p").set_values(price)
        ampl.get_parameter("EC_init").set_values([initial_capacity])

        # Specify the solver to use (e.g., HiGHS)
        ampl.option["solver"] = "gurobi"
        # Solve
        ampl.solve()

        schedule[day*24:(day+1)*24]  = ampl.get_variable('x').get_values().to_pandas()["x.val"]
        
        initial_capacity = round(initial_capacity + sum(schedule[day*24:(day+1)*24]))

    capacity = np.hstack((np.array([0]),np.cumsum(schedule)[:-1]))
    df_optim = df.copy()
    df_optim["schedule"] = schedule
    df_optim["capacity"] = np.hstack((np.array([0]),np.cumsum(schedule)[:-1]))
    df_optim["SOC"] = 100*df_optim["capacity"]/NEC

    return df_optim

        
        


# Plot functions :

In [48]:
def display_profit(df_optim) :
    """
    Displays daily profits 
    """

    days =pd.to_datetime(df_optim["timestamp"].apply(lambda x: datetime.datetime(x.year, x.month, x.day)),utc=True).unique()
    
    daily_profit = []


    for i in range(len(days)) :
        daily_profit.append(-df_optim.price_euros_wh.iloc[i*24:(i+1)*24] @ df_optim.schedule.iloc[i*24:(i+1)*24])

    fig = make_subplots(specs=[[{"secondary_y": True}]])


    fig.add_trace(go.Scatter(x=df_optim.timestamp, y=df_optim.price_euros_wh*10**6,name='Price (EUR/MWh)',line={"shape":"hv"},showlegend=True),
    secondary_y=False)

    fig.add_trace(go.Bar(x=days,y=daily_profit,name="Daily profit (EUR)",offset=2,showlegend=True,opacity=0.5),
    secondary_y=True)

    fig.update_layout(
    title_text="Daily profit<br>Total: {} EUR<br>Mean: {} EUR".format(int(sum(daily_profit)), int(np.mean(daily_profit)))
    )

    fig.update_xaxes(title_text="Hour")

    fig.update_yaxes(title_text="Price (EUR/MWh)", secondary_y=False)
    fig.update_yaxes(title_text="Daily profit (EUR)", secondary_y=True)

    fig.update_layout(bargap=0.)
    fig.write_html("out/profit.html")
    fig.show()
    


    
    



def display_schedule(df_optim,start,end) :
    """
    Displays charge schedule between start datetime and end datetime 
    """
		
    mask = (df_optim.timestamp<end) & (df_optim.timestamp>=start)
    df_to_show = df_optim[mask]


    fig = make_subplots(specs=[[{"secondary_y": True}]])


    fig.add_trace(go.Scatter(x=df_to_show.timestamp, y=df_to_show.price_euros_wh*10**6,name='Price (EUR/MWh)',line={"shape":"hv"},showlegend=True),
        secondary_y=False)

    fig.add_trace(
        go.Scatter(x=df_to_show.timestamp,y=df_to_show.SOC,name="SOC (%)",showlegend=True),secondary_y=True)



    color = None 
    count = 0

    shapes = []


    for h in df_to_show.index[np.sign(df_to_show.schedule).diff() != 0] : 
        
       
        if (df_to_show.schedule[h] == 0 and not color) or (df_to_show.schedule[h] > 0  and color == "green") or (df_to_show.schedule[h] < 0  and color == "red") :
            continue

        elif df_to_show.schedule[h] == 0 and color :
            shapes.append(dict(type="rect",x0=df_to_show.timestamp[start], y0=1, x1=df_to_show.timestamp[h], y1=100,  yref="y2",fillcolor=color, opacity=0.25, line_width=0))
            color = None

        elif df_to_show.schedule[h] > 0 and color == "red" : 
            shapes.append(dict(type="rect",x0=df_to_show.timestamp[start], y0=1, x1=df_to_show.timestamp[h], y1=100, yref="y2",fillcolor=color, opacity=0.25, line_width=0))
            start = h 
            color = "green"

        elif df_to_show.schedule[h] > 0 and not color :
            start = h
            color = "green"
            
        elif df_to_show.schedule[h] < 0 and color == "green" : 
            shapes.append(dict(type="rect",x0=df_to_show.timestamp[start], y0=1, x1=df_to_show.timestamp[h], y1=100, yref="y2", fillcolor=color, opacity=0.25, line_width=0))
            start = h 
            color = "red"
            
        elif df_to_show.schedule[h] < 0 and not color :
            start = h
            color = "red"






    # Add range slider
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(step="all"),
                    dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                    dict(count=2*7,
                        label="2w",
                        step="day",
                        stepmode="backward"),
                    dict(count=1*7,
                        label="1w",
                        step="day",
                        stepmode="backward"),
                    dict(count=2,
                        label="2d",
                        step="day",
                        stepmode="backward"),
                    dict(count=1,
                        label="1d",
                        step="day",
                        stepmode="backward"),
                    
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )

    fig.update_layout(
        shapes=shapes)


    fig.update_layout(
    title_text="Charge schedule.    Please use the buttons below to set the data range.<br>"
    )

    fig.update_xaxes(title_text="Date")

    fig.update_yaxes(title_text="Price (EUR/MWh)", secondary_y=False)
    fig.update_yaxes(title_text="SOC (%)", secondary_y=True)
        
    fig.show()  
    fig.write_html("out/schedule.html")





    

# Price dataset :

In [49]:
df = pd.read_csv("data/european_wholesale_electricity_price_data_hourly.csv")
df.rename(columns={"Datetime (UTC)":"timestamp","Price (EUR/MWhe)":"price_euros_wh"},inplace=True, errors='raise')
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%f%Z")

df.price_euros_wh /= 10 ** 6

start = "2022-01-15 00:00:00"
end = "2022-01-20 00:00:00"
country = "Germany"

df = df[df["Country"] == country]
mask = (df.timestamp<end) & (df.timestamp>=start)
df = df[mask]



# Optimize and show schedule :

In [56]:
# df_optim = optimize(df)
df_optim = optimize_day_by_day(df)

Gurobi 10.0.0:Gurobi 10.0.0: optimal solution; objective 11.2045
169 simplex iterations
1 branching nodes
Gurobi 10.0.0:Gurobi 10.0.0: optimal solution; objective 2.989
206 simplex iterations
1 branching nodes
Gurobi 10.0.0:Gurobi 10.0.0: optimal solution; objective 26.3715
220 simplex iterations
1 branching nodes
Gurobi 10.0.0:Gurobi 10.0.0: optimal solution; objective 14.198
174 simplex iterations
1 branching nodes
Gurobi 10.0.0:Gurobi 10.0.0: optimal solution; objective 11.327
185 simplex iterations
1 branching nodes


# Show profits :

In [57]:
start_show = start
end_show = end
display_schedule(df_optim,start_show,end_show)

In [58]:
display_profit(df_optim)